In [5]:
import os
import math
from docplex.mp.model import Model
from docplex.mp.relax_linear import LinearRelaxer

def fill_variables(file, n_customers, n_available_depots, coordinates_depots, coordinates_customers, v_capacity, capacities_depots,
        customers_demands, opening_costs_depots, opening_cost_route, zero_or_not):
    
    line = next(file)
    n_customers = int(line.strip())
    print("Número de customers: " + str(n_customers))
    line = next(file)
    n_available_depots = int(line.strip())
    print("Número de depots disponibles: " + str(n_available_depots))
    line = next(file)
    line = next(file)
            
    #coordenadas depots
    while(line.strip()):
        coordinates_depots.append(line.split())    
        line = next(file)
            
    #print("Coordenadas depots:")
    #print(coordinates_depots)
            
    line = next(file)
            
        #Coordenadas customers
    while(line.strip()):
        coordinates_customers.append(line.split())
                
        line = next(file)
            
    #print("Coordenadas customers:")
    #print(coordinates_customers)
            
    line = next(file)
            
    #Capacidad de vehículos
    v_capacity = int(line.strip())
    #print("Capacidad vehículos: " + v_capacity)
    line = next(file)
    line = next(file)
            
    #Capacidad de los depots
    while(line.strip()):
        capacities_depots.append(line.split())
                
        line = next(file)
            
    #print("Capacidades depots:")
    #print(capacities_depots)
            
    line = next(file)
            
    while(line.strip()):
        customers_demands.append(line.strip())
                
        line = next(file)
            
    #print("Demandas customers:")
    #print(customers_demands)
            
    line = next(file)
            
    while(line.strip()):
        opening_costs_depots.append(line.split())
                
        line = next(file)
            
    #print("Costos depots:")
    #print(opening_costs_depots)
            
    line = next(file)
            
            
    opening_cost_route = int(line.strip())
                
    line = next(file)
            
    #print("opening cost route:")
    #print(opening_cost_route)
            
    line = next(file)
    zero_or_not = int(line.strip())
            
    #print("zero o uno: " + zero_or_not)
    return [n_customers, n_available_depots, v_capacity, opening_cost_route, zero_or_not]
    
def distance_cost(a1, a2, b1, b2):
    a1 = int(a1)
    a2 = int(a2)
    b1 = int(b1)
    b2 = int(b2)
    x1 = a1-b1
    x2 = a2-b2
    x1 = pow(x1, 2)
    x2 = pow(x2, 2)
    return math.sqrt(x1 + x2)

In [19]:
file_path = "Inst1"
for filename in os.listdir(file_path):
    print("\nSiguiente archivo\n")
    f = os.path.join(file_path, filename)
    # checking if it is a file
    if os.path.isfile(f):
        
        #Contenedores de info inicializados en nada
        n_customers = 0
        n_available_depots = 0
        coordinates_depots = []
        coordinates_customers = []
        v_capacity = 0
        capacities_depots = []
        customers_demands = []
        opening_costs_depots = []
        opening_cost_route = 0
        zero_or_not = 1
        
        #Acá se toma la info de los archivos .dat
        with open(f, 'r') as file:
            lista_var = []
            lista_var = fill_variables(file, n_customers, n_available_depots, coordinates_depots, coordinates_customers, 
                                       v_capacity, capacities_depots, customers_demands, opening_costs_depots, 
                                       opening_cost_route, zero_or_not)
            n_customers = lista_var[0]
            n_available_depots = lista_var[1]
            v_capacity = lista_var[2]
            opening_cost_route = lista_var[3]
            zero_or_not = lista_var[4]
        
            
        #Empezar operaciones acá
        #distancias_d2d = []  VIAJE ENTRE 2 DEPOTS NO PERMITIDO
        distancias_d2c = []
        distancias_c2c = []
        distancias_c2d = []
        distancias = []
        
        #Acá se consiguen los c_ij (costos de caminos)
        for i in coordinates_depots:
            '''for j in coordinates_depots:
                if j==i:
                    continue
                distancias.append(distance_cost(i[0], i[1], j[0], j[1]))
                distancias_d2d.append(distance_cost(i[0], i[1], j[0], j[1]))'''
            for j in coordinates_customers:
                distancias.append(distance_cost(i[0], i[1], j[0], j[1]))
                distancias_d2c.append(distance_cost(i[0], i[1], j[0], j[1]))
        for i in coordinates_customers:
            for j in coordinates_depots:
                distancias.append(distance_cost(i[0], i[1], j[0], j[1]))
                distancias_c2d.append(distance_cost(i[0], i[1], j[0], j[1]))
        for i in coordinates_customers:
            for j in coordinates_customers:
                if j==i:
                    continue
                distancias.append(distance_cost(i[0], i[1], j[0], j[1]))
                distancias_c2c.append(distance_cost(i[0], i[1], j[0], j[1]))

        #MODELO
        mdl = Model(name="MDRP")
        x = mdl.binary_var_list(len(distancias), name="x")
        mdl.print_information()
        
        #print(len(distancias))
        #C_ij * X_ij
        ab = [distancias[i]*x[i] for i in range(n_customers*n_available_depots)]
        mdl.minimize(mdl.sum(ab))

        #X_dj = yd
        for i in range(n_available_depots):
            offset = i*n_customers
            xdj = []
            for j in range(offset, offset+n_customers):
                xdj.append(x[j])
            mdl.add_constraint(sum(xdj) <= 1)
            mdl.add_constraint(sum(xdj) >= 0)
            
        #X_jd = yd
        for i in range(n_available_depots):
            xjd = []
            for j in range(n_customers):
                offset = j*n_available_depots + n_available_depots*n_customers
                #print(str(offset) + " vs " + str(len(distancias)) + " vs " + str(len(x)))
                xjd.append(x[offset])
            mdl.add_constraint(sum(xjd) <= 1)
            mdl.add_constraint(sum(xjd) >= 0)
            
        
            
        


Siguiente archivo

Número de customers: 100
Número de depots disponibles: 10
Model: MDRP
 - number of variables: 11898
   - binary=11898, integer=0, continuous=0
 - number of constraints: 0
   - linear=0
 - parameters: defaults
 - objective: none
 - problem type is: MILP
x_1000+x_1010+x_1020+x_1030+x_1040+x_1050+x_1060+x_1070+x_1080+x_1090+x_1100+x_1110+x_1120+x_1130+x_1140+x_1150+x_1160+x_1170+x_1180+x_1190+x_1200+x_1210+x_1220+x_1230+x_1240+x_1250+x_1260+x_1270+x_1280+x_1290+x_1300+x_1310+x_1320+x_1330+x_1340+x_1350+x_1360+x_1370+x_1380+x_1390+x_1400+x_1410+x_1420+x_1430+x_1440+x_1450+x_1460+x_1470+x_1480+x_1490+x_1500+x_1510+x_1520+x_1530+x_1540+x_1550+x_1560+x_1570+x_1580+x_1590+x_1600+x_1610+x_1620+x_1630+x_1640+x_1650+x_1660+x_1670+x_1680+x_1690+x_1700+x_1710+x_1720+x_1730+x_1740+x_1750+x_1760+x_1770+x_1780+x_1790+x_1800+x_1810+x_1820+x_1830+x_1840+x_1850+x_1860+x_1870+x_1880+x_1890+x_1900+x_1910+x_1920+x_1930+x_1940+x_1950+x_1960+x_1970+x_1980+x_1990 <= 1
x_1000+x_1010+x_1020+x

Model: MDRP
 - number of variables: 11896
   - binary=11896, integer=0, continuous=0
 - number of constraints: 0
   - linear=0
 - parameters: defaults
 - objective: none
 - problem type is: MILP
x_1000+x_1010+x_1020+x_1030+x_1040+x_1050+x_1060+x_1070+x_1080+x_1090+x_1100+x_1110+x_1120+x_1130+x_1140+x_1150+x_1160+x_1170+x_1180+x_1190+x_1200+x_1210+x_1220+x_1230+x_1240+x_1250+x_1260+x_1270+x_1280+x_1290+x_1300+x_1310+x_1320+x_1330+x_1340+x_1350+x_1360+x_1370+x_1380+x_1390+x_1400+x_1410+x_1420+x_1430+x_1440+x_1450+x_1460+x_1470+x_1480+x_1490+x_1500+x_1510+x_1520+x_1530+x_1540+x_1550+x_1560+x_1570+x_1580+x_1590+x_1600+x_1610+x_1620+x_1630+x_1640+x_1650+x_1660+x_1670+x_1680+x_1690+x_1700+x_1710+x_1720+x_1730+x_1740+x_1750+x_1760+x_1770+x_1780+x_1790+x_1800+x_1810+x_1820+x_1830+x_1840+x_1850+x_1860+x_1870+x_1880+x_1890+x_1900+x_1910+x_1920+x_1930+x_1940+x_1950+x_1960+x_1970+x_1980+x_1990 <= 1
x_1000+x_1010+x_1020+x_1030+x_1040+x_1050+x_1060+x_1070+x_1080+x_1090+x_1100+x_1110+x_1120+x_1130+x_

Model: MDRP
 - number of variables: 11886
   - binary=11886, integer=0, continuous=0
 - number of constraints: 0
   - linear=0
 - parameters: defaults
 - objective: none
 - problem type is: MILP
x_1000+x_1010+x_1020+x_1030+x_1040+x_1050+x_1060+x_1070+x_1080+x_1090+x_1100+x_1110+x_1120+x_1130+x_1140+x_1150+x_1160+x_1170+x_1180+x_1190+x_1200+x_1210+x_1220+x_1230+x_1240+x_1250+x_1260+x_1270+x_1280+x_1290+x_1300+x_1310+x_1320+x_1330+x_1340+x_1350+x_1360+x_1370+x_1380+x_1390+x_1400+x_1410+x_1420+x_1430+x_1440+x_1450+x_1460+x_1470+x_1480+x_1490+x_1500+x_1510+x_1520+x_1530+x_1540+x_1550+x_1560+x_1570+x_1580+x_1590+x_1600+x_1610+x_1620+x_1630+x_1640+x_1650+x_1660+x_1670+x_1680+x_1690+x_1700+x_1710+x_1720+x_1730+x_1740+x_1750+x_1760+x_1770+x_1780+x_1790+x_1800+x_1810+x_1820+x_1830+x_1840+x_1850+x_1860+x_1870+x_1880+x_1890+x_1900+x_1910+x_1920+x_1930+x_1940+x_1950+x_1960+x_1970+x_1980+x_1990 <= 1
x_1000+x_1010+x_1020+x_1030+x_1040+x_1050+x_1060+x_1070+x_1080+x_1090+x_1100+x_1110+x_1120+x_1130+x_

Model: MDRP
 - number of variables: 10896
   - binary=10896, integer=0, continuous=0
 - number of constraints: 0
   - linear=0
 - parameters: defaults
 - objective: none
 - problem type is: MILP
x_500+x_505+x_510+x_515+x_520+x_525+x_530+x_535+x_540+x_545+x_550+x_555+x_560+x_565+x_570+x_575+x_580+x_585+x_590+x_595+x_600+x_605+x_610+x_615+x_620+x_625+x_630+x_635+x_640+x_645+x_650+x_655+x_660+x_665+x_670+x_675+x_680+x_685+x_690+x_695+x_700+x_705+x_710+x_715+x_720+x_725+x_730+x_735+x_740+x_745+x_750+x_755+x_760+x_765+x_770+x_775+x_780+x_785+x_790+x_795+x_800+x_805+x_810+x_815+x_820+x_825+x_830+x_835+x_840+x_845+x_850+x_855+x_860+x_865+x_870+x_875+x_880+x_885+x_890+x_895+x_900+x_905+x_910+x_915+x_920+x_925+x_930+x_935+x_940+x_945+x_950+x_955+x_960+x_965+x_970+x_975+x_980+x_985+x_990+x_995 <= 1
x_500+x_505+x_510+x_515+x_520+x_525+x_530+x_535+x_540+x_545+x_550+x_555+x_560+x_565+x_570+x_575+x_580+x_585+x_590+x_595+x_600+x_605+x_610+x_615+x_620+x_625+x_630+x_635+x_640+x_645+x_650+x_655+x_660+x_

Model: MDRP
 - number of variables: 10900
   - binary=10900, integer=0, continuous=0
 - number of constraints: 0
   - linear=0
 - parameters: defaults
 - objective: none
 - problem type is: MILP
x_500+x_505+x_510+x_515+x_520+x_525+x_530+x_535+x_540+x_545+x_550+x_555+x_560+x_565+x_570+x_575+x_580+x_585+x_590+x_595+x_600+x_605+x_610+x_615+x_620+x_625+x_630+x_635+x_640+x_645+x_650+x_655+x_660+x_665+x_670+x_675+x_680+x_685+x_690+x_695+x_700+x_705+x_710+x_715+x_720+x_725+x_730+x_735+x_740+x_745+x_750+x_755+x_760+x_765+x_770+x_775+x_780+x_785+x_790+x_795+x_800+x_805+x_810+x_815+x_820+x_825+x_830+x_835+x_840+x_845+x_850+x_855+x_860+x_865+x_870+x_875+x_880+x_885+x_890+x_895+x_900+x_905+x_910+x_915+x_920+x_925+x_930+x_935+x_940+x_945+x_950+x_955+x_960+x_965+x_970+x_975+x_980+x_985+x_990+x_995 <= 1
x_500+x_505+x_510+x_515+x_520+x_525+x_530+x_535+x_540+x_545+x_550+x_555+x_560+x_565+x_570+x_575+x_580+x_585+x_590+x_595+x_600+x_605+x_610+x_615+x_620+x_625+x_630+x_635+x_640+x_645+x_650+x_655+x_660+x_

...

Model: MDRP
 - number of variables: 43764
   - binary=43764, integer=0, continuous=0
 - number of constraints: 0
   - linear=0
 - parameters: defaults
 - objective: none
 - problem type is: MILP
x_2000+x_2010+x_2020+x_2030+x_2040+x_2050+x_2060+x_2070+x_2080+x_2090+x_2100+x_2110+x_2120+x_2130+x_2140+x_2150+x_2160+x_2170+x_2180+x_2190+x_2200+x_2210+x_2220+x_2230+x_2240+x_2250+x_2260+x_2270+x_2280+x_2290+x_2300+x_2310+x_2320+x_2330+x_2340+x_2350+x_2360+x_2370+x_2380+x_2390+x_2400+x_2410+x_2420+x_2430+x_2440+x_2450+x_2460+x_2470+x_2480+x_2490+x_2500+x_2510+x_2520+x_2530+x_2540+x_2550+x_2560+x_2570+x_2580+x_2590+x_2600+x_2610+x_2620+x_2630+x_2640+x_2650+x_2660+x_2670+x_2680+x_2690+x_2700+x_2710+x_2720+x_2730+x_2740+x_2750+x_2760+x_2770+x_2780+x_2790+x_2800+x_2810+x_2820+x_2830+x_2840+x_2850+x_2860+x_2870+x_2880+x_2890+x_2900+x_2910+x_2920+x_2930+x_2940+x_2950+x_2960+x_2970+x_2980+x_2990+x_3000+x_3010+x_3020+x_3030+x_3040+x_3050+x_3060+x_3070+x_3080+x_3090+x_3100+x_3110+x_3120+x_3130+x_3140+

Model: MDRP
 - number of variables: 43780
   - binary=43780, integer=0, continuous=0
 - number of constraints: 0
   - linear=0
 - parameters: defaults
 - objective: none
 - problem type is: MILP
x_2000+x_2010+x_2020+x_2030+x_2040+x_2050+x_2060+x_2070+x_2080+x_2090+x_2100+x_2110+x_2120+x_2130+x_2140+x_2150+x_2160+x_2170+x_2180+x_2190+x_2200+x_2210+x_2220+x_2230+x_2240+x_2250+x_2260+x_2270+x_2280+x_2290+x_2300+x_2310+x_2320+x_2330+x_2340+x_2350+x_2360+x_2370+x_2380+x_2390+x_2400+x_2410+x_2420+x_2430+x_2440+x_2450+x_2460+x_2470+x_2480+x_2490+x_2500+x_2510+x_2520+x_2530+x_2540+x_2550+x_2560+x_2570+x_2580+x_2590+x_2600+x_2610+x_2620+x_2630+x_2640+x_2650+x_2660+x_2670+x_2680+x_2690+x_2700+x_2710+x_2720+x_2730+x_2740+x_2750+x_2760+x_2770+x_2780+x_2790+x_2800+x_2810+x_2820+x_2830+x_2840+x_2850+x_2860+x_2870+x_2880+x_2890+x_2900+x_2910+x_2920+x_2930+x_2940+x_2950+x_2960+x_2970+x_2980+x_2990+x_3000+x_3010+x_3020+x_3030+x_3040+x_3050+x_3060+x_3070+x_3080+x_3090+x_3100+x_3110+x_3120+x_3130+x_3140+

Model: MDRP
 - number of variables: 43780
   - binary=43780, integer=0, continuous=0
 - number of constraints: 0
   - linear=0
 - parameters: defaults
 - objective: none
 - problem type is: MILP
x_2000+x_2010+x_2020+x_2030+x_2040+x_2050+x_2060+x_2070+x_2080+x_2090+x_2100+x_2110+x_2120+x_2130+x_2140+x_2150+x_2160+x_2170+x_2180+x_2190+x_2200+x_2210+x_2220+x_2230+x_2240+x_2250+x_2260+x_2270+x_2280+x_2290+x_2300+x_2310+x_2320+x_2330+x_2340+x_2350+x_2360+x_2370+x_2380+x_2390+x_2400+x_2410+x_2420+x_2430+x_2440+x_2450+x_2460+x_2470+x_2480+x_2490+x_2500+x_2510+x_2520+x_2530+x_2540+x_2550+x_2560+x_2570+x_2580+x_2590+x_2600+x_2610+x_2620+x_2630+x_2640+x_2650+x_2660+x_2670+x_2680+x_2690+x_2700+x_2710+x_2720+x_2730+x_2740+x_2750+x_2760+x_2770+x_2780+x_2790+x_2800+x_2810+x_2820+x_2830+x_2840+x_2850+x_2860+x_2870+x_2880+x_2890+x_2900+x_2910+x_2920+x_2930+x_2940+x_2950+x_2960+x_2970+x_2980+x_2990+x_3000+x_3010+x_3020+x_3030+x_3040+x_3050+x_3060+x_3070+x_3080+x_3090+x_3100+x_3110+x_3120+x_3130+x_3140+

Model: MDRP
 - number of variables: 2950
   - binary=2950, integer=0, continuous=0
 - number of constraints: 0
   - linear=0
 - parameters: defaults
 - objective: none
 - problem type is: MILP
x_250+x_255+x_260+x_265+x_270+x_275+x_280+x_285+x_290+x_295+x_300+x_305+x_310+x_315+x_320+x_325+x_330+x_335+x_340+x_345+x_350+x_355+x_360+x_365+x_370+x_375+x_380+x_385+x_390+x_395+x_400+x_405+x_410+x_415+x_420+x_425+x_430+x_435+x_440+x_445+x_450+x_455+x_460+x_465+x_470+x_475+x_480+x_485+x_490+x_495 <= 1
x_250+x_255+x_260+x_265+x_270+x_275+x_280+x_285+x_290+x_295+x_300+x_305+x_310+x_315+x_320+x_325+x_330+x_335+x_340+x_345+x_350+x_355+x_360+x_365+x_370+x_375+x_380+x_385+x_390+x_395+x_400+x_405+x_410+x_415+x_420+x_425+x_430+x_435+x_440+x_445+x_450+x_455+x_460+x_465+x_470+x_475+x_480+x_485+x_490+x_495 <= 1
x_250+x_255+x_260+x_265+x_270+x_275+x_280+x_285+x_290+x_295+x_300+x_305+x_310+x_315+x_320+x_325+x_330+x_335+x_340+x_345+x_350+x_355+x_360+x_365+x_370+x_375+x_380+x_385+x_390+x_395+x_400+x_405+x_410

Model: MDRP
 - number of variables: 2944
   - binary=2944, integer=0, continuous=0
 - number of constraints: 0
   - linear=0
 - parameters: defaults
 - objective: none
 - problem type is: MILP
x_250+x_255+x_260+x_265+x_270+x_275+x_280+x_285+x_290+x_295+x_300+x_305+x_310+x_315+x_320+x_325+x_330+x_335+x_340+x_345+x_350+x_355+x_360+x_365+x_370+x_375+x_380+x_385+x_390+x_395+x_400+x_405+x_410+x_415+x_420+x_425+x_430+x_435+x_440+x_445+x_450+x_455+x_460+x_465+x_470+x_475+x_480+x_485+x_490+x_495 <= 1
x_250+x_255+x_260+x_265+x_270+x_275+x_280+x_285+x_290+x_295+x_300+x_305+x_310+x_315+x_320+x_325+x_330+x_335+x_340+x_345+x_350+x_355+x_360+x_365+x_370+x_375+x_380+x_385+x_390+x_395+x_400+x_405+x_410+x_415+x_420+x_425+x_430+x_435+x_440+x_445+x_450+x_455+x_460+x_465+x_470+x_475+x_480+x_485+x_490+x_495 <= 1
x_250+x_255+x_260+x_265+x_270+x_275+x_280+x_285+x_290+x_295+x_300+x_305+x_310+x_315+x_320+x_325+x_330+x_335+x_340+x_345+x_350+x_355+x_360+x_365+x_370+x_375+x_380+x_385+x_390+x_395+x_400+x_405+x_410